In [2]:
pip install pdfplumber

  Using cached pdfplumber-0.11.4-py3-none-any.whl.metadata (41 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
  Using cached pypdfium2-4.30.0-py3-none-win_amd64.whl.metadata (48 kB)
Using cached pdfplumber-0.11.4-py3-none-any.whl (59 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
Using cached pypdfium2-4.30.0-py3-none-win_amd64.whl (2.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [18]:
import pdfplumber
def extract_with_pdf(path):
    text =""
    with pdfplumber.open(path) as pdfreader:
        for page in pdfreader.pages:
            text+=page.extract_text() or ""
    return text


In [19]:
text_extracted=extract_with_pdf(r"D:\waste for git\Generative_AI\Rag\attention all you need.pdf")
#print(text_extracted)

In [13]:
pip install Langchain


Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS

In [31]:
# dividing into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,         # Maximum size of each chunk
        chunk_overlap=50,       # Overlap between chunks
        length_function=len,    # Function to calculate length
        separators=["\n\n", "\n", ". ", " ", ""]  # Possible split points
    )
    chunks = text_splitter.split_text(text)
    return chunks

# Example usage
text_chunks = get_text_chunks(text_extracted)
print(text_chunks)
print(len(text_extracted))

#get_text_chunks(text_extracted)

['Attention Is All You Need\nAshishVaswani∗ NoamShazeer∗ NikiParmar∗ JakobUszkoreit∗\nGoogleBrain GoogleBrain GoogleResearch GoogleResearch\navaswani@google.com noam@google.com nikip@google.com usz@google.com\nLlionJones∗ AidanN.Gomez∗ † ŁukaszKaiser∗\nGoogleResearch UniversityofToronto GoogleBrain\nllion@google.com aidan@cs.toronto.edu lukaszkaiser@google.com\nIlliaPolosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThedominantsequencetransductionmodelsarebasedoncomplexrecurrentor', 'convolutionalneuralnetworksthatincludeanencoderandadecoder. Thebest\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbasedsolelyonattentionmechanisms,dispensingwithrecurrenceandconvolutions\nentirely. Experiments on two machine translation tasks show these models to\nbesuperiorinqualitywhilebeingmoreparallelizableandrequiringsignificantly', 'less time to train. Our model achieves 28.4 BLEU on the WMT 2

In [33]:
pip install faiss-cpu


   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.8/14.9 MB 11.2 MB/s eta 0:00:02
   ----------- ---------------------------- 4.2/14.9 MB 11.4 MB/s eta 0:00:01
   ----------------- ---------------------- 6.6/14.9 MB 11.2 MB/s eta 0:00:01
   ----------------------- ---------------- 8.9/14.9 MB 11.3 MB/s eta 0:00:01
   ------------------------------ --------- 11.3/14.9 MB 11.2 MB/s eta 0:00:01
   ------------------------------------ --- 13.6/14.9 MB 11.1 MB/s eta 0:00:01
   ---------------------------------------  14.7/14.9 MB 11.1 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 9.3 MB/s eta 0:00:00


In [34]:
# embeddings and storing
def get_vector_store(text_chunks):
    embeddings=HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_store=FAISS.from_texts(text_chunks,embedding=embeddings)
    vector_store.save_local("faiss_index")
    
get_vector_store(text_chunks)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama

def create_qa_chain():
    prompt_template = """
    Use the following pieces of context to answer the question. If you cannot find the answer in the context, respond with "The answer is not available in the context." Do not make up or infer any information that is not explicitly stated in the context.

    Context: {context}

    Question: {question}

    Instructions:
    1. Only use information from the provided context
    2. If the exact answer is in the context, provide it
    3. If the answer is not in the context, say "The answer is not available in the context"
    4. Keep the answer concise and relevant

    Answer:
    """

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    llm = Ollama(model="llama3.1", temperature=0.3)  # Load the Llama model

    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.load_local("faiss_index", embeddings)

    retriever = vectorstore.as_retriever(
        search_type="similarity",  # Use similarity search to retrieve relevant chunks
        search_kwargs={
            "k": 5,                   # Number of relevant chunks to retrieve
            "fetch_k": 15             # Total chunks to fetch
        }
    )

    # Create a QA chain that combines LLM with the retriever
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Use the "stuff" type for this chain
        retriever=retriever,
        chain_type_kwargs={
            "prompt": PROMPT,
        },
        return_source_documents=True  # Optionally return source documents for transparency
    )
    
    return qa_chain

# Example usage
qa_chain = create_qa_chain()


In [ ]:
# Memory

class Memory:
    def __init__(self):
        self.conversations = []

    def add_conversation(self, user_input, bot_response):
        self.conversations.append({'user': user_input, 'bot': bot_response})

    def get_context(self):
        return "\n".join([f"User: {conv['user']}\nBot: {conv['bot']}" for conv in self.conversations])

# Example usage in handle_user_input
memory = Memory()

def handle_user_input(user_question):
    context = memory.get_context()
    try:
        qa_chain = create_qa_chain()  # Create the QA chain
        response = qa_chain.invoke({
            "query": f"{context}\nUser: {user_question}"  # Include memory context
        })
        
        answer = response.get('result', '').strip()  # Extract the answer
        memory.add_conversation(user_question, answer)  # Store the conversation

        st.write("Reply: ", answer)

    except Exception as e:
        st.error(f"An error occurred: {str(e)}")
        st.write("Reply: The answer is not available in the context.")
